# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Zhang  ->  R. Zhang  |  ['Zhang']
J. Liu  ->  J. Liu  |  ['J. Liu']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
Arxiv has 49 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2411.09729


extracting tarball to tmp_2411.09729...

 done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2411.09729/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authors' from 'tmp_2411.09729/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

extracting tarball to tmp_2411.09753...

 done.
  0: tmp_2411.09753/natnotes.tex, 332 lines
  1: tmp_2411.09753/main.tex, 854 lines
  2: tmp_2411.09753/natbib.tex, 96 lines
  3: tmp_2411.09753/aassymbols.tex, 579 lines
Retrieving document from  https://arxiv.org/e-print/2411.09905


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 4 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2411.09753/main.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2411.09905...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.09729-b31b1b.svg)](https://arxiv.org/abs/2411.09729) | **The SDSS-V Local Volume Mapper (LVM): Data Analysis Pipeline**  |
|| S. F. Sanchez, et al. -- incl., <mark>K. Kreckel</mark> |
|*Appeared on*| *2024-11-18*|
|*Comments*| *35 pages, 17 figures, 8 tables, accepted for publication in AJ*|
|**Abstract**|            We introduce the Data Analysis Pipeline (DAP) for the Sloan Digital Sky Survey V (SDSS-V) Local Volume Mapper (LVM) project, referred to as the LVM-DAP. We outline our methods for recovering both stellar and emission line components from the optical integral field spectroscopy, highlighting the developments and changes implemented to address specific challenges of the data set. The observations from the LVM project are unique because they cover a wide range of physical resolutions, from approximately 0.05 pc to 100 pc, depending on the distance to the targets. This, along with the varying number of stars sampled in each aperture (ranging from zero, just one of a few, to thousands), presents challenges in using previous spectral synthesis methods and interpreting the spectral fits. We provide a detailed explanation of how we model the stellar content and separate it from the ionized gas emission lines. To assess the accuracy of our results, we compare them with both idealized and more realistic simulations, highlighting the limitations of our methods. We find that the DAP robustly correct for stellar continuum features and recover emission line parameters (e.g. flux, equivalent width, systemtic velocity and velocity dispersion) with a precision and accuracy that fulfill the requirements of the primary goal of the analysis. In addition, the recovered stellar parameters are reliable for single stars, the recovery of integrated populations is less precise. We conclude with a description of the data products we provide, instructions for downloading and using our software, and a showcase illustrating the quality of the data and the analysis on a deep exposure taken on the Huygens region at the center of the Orion Nebula.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.09735-b31b1b.svg)](https://arxiv.org/abs/2411.09735) | **Gas thermodynamics meets galaxy kinematics: Joint mass measurements for eROSITA galaxy clusters**  |
|| P. Li, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-11-18*|
|*Comments*| *14 pages, 9 figures, 2 tables, with 12-page appendix for additional figures and tables. Accepted for publication in A&A*|
|**Abstract**|            The mass of galaxy clusters is a critical quantity for probing cluster cosmology and testing theories of gravity, but its measurement could be biased given assumptions are inevitable. In this paper, we employ and compare two mass proxies for galaxy clusters: thermodynamics of the intracluster medium and kinematics of member galaxies. We select 22 galaxy clusters from the cluster catalog in the first SRG/eROSITA All-Sky Survey (eRASS1) that have sufficient optical and near-infrared observations. We generate multi-band images in the energy range of (0.3, 7) keV for each cluster, and derive their temperature profiles, gas mass profiles and hydrostatic mass profiles using a parametric approach that does not assume dark matter halo models. With spectroscopically confirmed member galaxies collected from multiple surveys, we numerically solve the spherical Jeans equation for their dynamical mass profiles. Our results quantify the correlation between dynamical mass and line-of-sight velocity dispersion with an rms scatter of 0.14 dex. We find the two mass proxies lead to roughly the same total mass, with no observed systematic bias. As such, the $\sigma_8$ tension is not specific to hydrostatic mass or weak lensing shears, but also appears with galaxy kinematics. We also compare our hydrostatic masses with the latest weak lensing masses inferred with scaling relations. The comparison shows the weak lensing mass is significantly higher than our hydrostatic mass by $\sim$110%. This might explain the significantly larger value of $\sigma_8$ from the latest measurement using eRASS1 clusters than almost all previous estimates in the literature. Finally, we test the radial acceleration relation (RAR) established in disk galaxies. We confirm the missing baryon problem in the inner region of galaxy clusters using three independent mass proxies for the first time.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.09753-b31b1b.svg)](https://arxiv.org/abs/2411.09753) | **The democratic detrender: Ensemble-Based Removal of the Nuisance Signal in Stellar Time-Series Photometry**  |
|| D. A. Yahalomi, et al. -- incl., <mark>Zhang</mark>, <mark>J. Liu</mark> |
|*Appeared on*| *2024-11-18*|
|*Comments*| *15 pages, 10 figures, submitted to ApJS. For source code, documentation, and tutorials, see this https URL*|
|**Abstract**|            Accurate, precise, and computationally efficient removal of unwanted activity that exists as a combination of periodic, quasi-periodic, and non-periodic systematic trends in time-series photometric data is a critical step in exoplanet transit analysis. Throughout the years, many different modeling methods have been used for this process, often called "detrending." However, there is no community-wide consensus regarding the favored approach. In order to mitigate model dependency, we present an ensemble-based approach to detrending via community-of-models and the $\texttt{democratic detrender}$: a modular and scalable open-source coding package that implements ensemble detrending. The $\texttt{democratic detrender}$ allows users to select from a number of packaged detrending methods (including cosine filtering, Gaussian processes, and polynomial fits) or provide their own set of detrended light curves via methods of their choosing. The $\texttt{democratic detrender}$ then combines the resulting individually detrended light curves into a single method marginalized (via median selection) light curve. Additionally, the $\texttt{democratic detrender}$ inflates the uncertainties of each time-series data point using information from the median absolute deviation between the individually detrended light curve, propagating information into the final detrended light curve about the added uncertainty due to the correctness of the underlying models.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.09905-b31b1b.svg)](https://arxiv.org/abs/2411.09905) | **Eccentricities of Close Stellar Binaries**  |
|| Y. Wu, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2024-11-18*|
|*Comments*| *6 pages + 7 appendixes; submitted to ApJ Letters*|
|**Abstract**|            Orbits of stellar binaries are in general eccentric. This encodes information about the formation process. Here, we use thousands of main-sequence binaries from the GAIA DR3 catalog to reveal that, binaries inwards of a few AU exhibit a simple Rayleigh distribution with a mode $\sigma_e \simeq 0.30$. We find the same distribution for binaries from M to A spectral types, and from tens of days to $10^3$days (possibly extending to tens of AU). This observed distribution is most likely primordial. Its Rayleigh form suggests an origin in weak scattering, while its invariant mode demands a universal process. We experiment with exciting binary eccentricities by ejecting brown dwarfs, and find that the eccentricities reach an equi-partition value of $\sigma_e \simeq \sqrt{M_{\rm bd}/M_*}$. So to explain the observed mode, these brown dwarfs will have to be of order one tenth the stellar masses, and be at least as abundant in the Galaxy as the close binaries. The veracity of such a proposal remains to be tested.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2411.09729/./figures/m_sel-ORION.png', 'tmp_2411.09729/./figures/Scene2_sim_in.png', 'tmp_2411.09729/./figures/Scene2_sim_out.png', 'tmp_2411.09729/./figures/m_fit_Scene1.png', 'tmp_2411.09729/./figures/Rosetta_sim_in.png', 'tmp_2411.09729/./figures/Rosetta_sim_out.png', 'tmp_2411.09729/./figures/m_fit_00006109_cl_SN1_342_5_5_20.0.png']
copying  tmp_2411.09729/./figures/m_sel-ORION.png to _build/html/
copying  tmp_2411.09729/./figures/Scene2_sim_in.png to _build/html/
copying  tmp_2411.09729/./figures/Scene2_sim_out.png to _build/html/
copying  tmp_2411.09729/./figures/m_fit_Scene1.png to _build/html/
copying  tmp_2411.09729/./figures/Rosetta_sim_in.png to _build/html/
copying  tmp_2411.09729/./figures/Rosetta_sim_out.png to _build/html/
copying  tmp_2411.09729/./figures/m_fit_00006109_cl_SN1_342_5_5_20.0.png to _build/html/
exported in  _build/html/2411.09729.md
    + _build/html/tmp_2411.09729/./figures/m_sel-ORION.png
    + _build/html/tmp_2411.09729/./figures/Sc

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\lam}{\lambda}$
$\newcommand{\hi}{\ION{H}{i}}$
$\newcommand{\hii}{\ION{H}{ii}}$
$\newcommand{\hh}{\ION{H}{ii}~}$
$\newcommand{\hei}{\ION{He}{i}}$
$\newcommand{\heii}{\ION{He}{ii}}$
$\newcommand{\nii}{[\ION{N}{ii}]}$
$\newcommand{\oi}{[\ION{O}{i}]}$
$\newcommand{\oii}{[\ION{O}{ii}]}$
$\newcommand{\oiii}{[\ION{O}{iii}]}$
$\newcommand{\sii}{[\ION{S}{ii}]}$
$\newcommand{\siii}{[\ION{S}{iii}]}$
$\newcommand{\caii}{[\ION{Ca}{ii}]}$
$\newcommand{\feii}{[\ION{Fe}{ii}]}$
$\newcommand{\ariii}{[\ION{Ar}{iii}]}$
$\newcommand{\neiii}{[\ION{Ne}{iii}]}$
$\newcommand{\eduardo}[1]{\textbf{\textcolor{blue}{Eduardo: {#1}}}}$
$\newcommand{\ha}{\rm{H}\alpha}$
$\newcommand{\hb}{\rm{H}\beta}$
$\newcommand{\hd}{\rm{H}\delta}$
$\newcommand{\hg}{\rm{H}\gamma}$
$\newcommand{\he}{\rm{H}\epsilon}$
$\newcommand{\mHa}{\rm{H}\alpha}$
$\newcommand{\Ha}{\rm{H}\alpha}$
$\newcommand{\Hb}{\rm{H}\beta}$
$\newcommand{\logU}{\log(\bar U)}$
$\newcommand{\Kdist}{K_{dist}}$
$\newcommand{\EWa}{EW(\Ha)}$
$\newcommand{\fOB}{f(OB)}$
$\newcommand{\QHHe}{Q_{0/1}}$
$\newcommand{\Com}[1]{{\color{red}*** #1}}$
$\newcommand{\Te}{T_{\rm e}}$
$\newcommand{\Ssfr}{\Sigma_{\rm SFR} }$
$\newcommand{\Sst}{\Sigma_{\rm *} }$
$\newcommand{\Sgas}{\Sigma_{\rm mol} }$
$\newcommand{\Srgas}{\Sigma_{\rm gas} }$
$\newcommand{\eSsfr}{\Sigma_{\rm SFR}}$
$\newcommand{\eSst}{\Sigma_{\rm *}}$
$\newcommand{\eSgas}{\Sigma_{\rm mol}}$
$\newcommand{\Rsfr}{\rho_{\rm SFR} }$
$\newcommand{\Rst}{\rho_{\rm *} }$
$\newcommand{\Rgas}{\rho_{\rm gas} }$
$\newcommand{\alf}{[\alpha/Fe]}$
$\newcommand{\oz}{[O/Fe]}$
$\newcommand{\fe}{[Fe/H]}$
$\newcommand{\age}{\mathcal{A}_{\star,L}}$
$\newcommand{\met}{\mathcal{Z}_{\star,L}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\flux}{erg s^{-1} cm^{-2}}$
$\newcommand{\edr}{Sánchez et al. (submitted)}$
$\newcommand{\edrp}{(Sánchez et al., submitted)}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\OSU}{\label{OSU} Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}$
$\newcommand{\Alberta}{\label{Alberta} Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\newcommand{\ANU}{\label{ANU} Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\IPAC}{\label{IPAC} Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}$
$\newcommand{\Carnegie}{\label{Carnegie} Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}$
$\newcommand{ÇAPP}{\label{CCAPP} Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\CfA}{\label{CfA}Harvard-Smithsonian Center for Astrophysics, 60 Garden Street, Cambridge, MA 02138, USA}$
$\newcommand{\CITEVA}{\label{CITEVA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{\CNRS}{\label{CNRS} CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}$
$\newcommand{\ESO}{\label{ESO} European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}$
$\newcommand{\ESOChile}{\label{ESOChile} European Southern Observatory, Avenida Alonso de Cordoba 3107, Casilla 19, Santiago 19001, Chile}$
$\newcommand{\HD}{\label{HD} Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{\ICRAR}{\label{ICRAR} International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}$
$\newcommand{\IRAM}{\label{IRAM} Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}$
$\newcommand{\ITA}{\label{ITA} Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\newcommand{\IWR}{\label{IWR} Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{\JHU}{\label{JHU} Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\newcommand{\Leiden}{\label{Leiden} Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}$
$\newcommand{\Maryland}{\label{Maryland} Department of Astronomy, University of Maryland, College Park, MD 20742, USA}$
$\newcommand{\MPE}{\label{MPE} Max-Planck-Institut für extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}$
$\newcommand{\MPIA}{\label{MPIA} Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{\Nagoya}{\label{Nagoya} Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}$
$\newcommand{\NRAO}{\label{NRAO} National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}$
$\newcommand{\OAN}{\label{OAN} Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{\ObsParis}{\label{ObsParis} Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}$
$\newcommand{\Princeton}{\label{Princeton} Department of Astrophysical Sciences, Princeton University, Princeton, NJ 08544 USA}$
$\newcommand{\UToledo}{\label{UToledo} University of Toledo, 2801 W. Bancroft St., Mail Stop 111, Toledo, OH, 43606}$
$\newcommand{\Toulouse}{\label{Toulouse} Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}$
$\newcommand{\UBonn}{\label{UBonn} Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}$
$\newcommand{\UChile}{\label{UChile} Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}$
$\newcommand{\UConn}{\label{UConn} Department of Physics, University of Connecticut, Storrs, CT, 06269, USA}$
$\newcommand{\UCSD}{\label{UCSD}Center for Astrophysics and Space Sciences, Department of Physics,  University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\newcommand{\UGent}{\label{UGent} Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\ULyon}{\label{ULyon} Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574,\ F-69230 Saint-Genis-Laval, France}$
$\newcommand{\UMass}{\label{UMass} University of Massachusetts—Amherst, 710 N. Pleasant Street, Amherst, MA 01003, USA}$
$\newcommand{\UWyoming}{\label{UWyoming} Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\newcommand{\LAM}{\label{LAM} Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille), Marseille, France}$
$\newcommand{\UHawaii}{\label{UHawaii} Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}$
$\newcommand{\UCM}{\label{UCM} Departamento de Física de la Tierra y Astrofísica, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\IPARC}{\label{IPARC} Instituto de Física de Partículas y del Cosmos IPARCOS, Facultad de Ciencias Físicas, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\STScI}{\label{STScI} Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\McMaster}{\label{McMaster} Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\newcommand{\INAF}{\label{INAF} INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Firenze, Italy}$
$\newcommand{\Sydney}{\label{Sydney} Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}$
$\newcommand{\CITA}{\label{CITA} Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George St, Toronto, ON M5S 3H8, Canada}$
$\newcommand{\ASIAA}{\label{ASIAA} Institute of Astronomy and Astrophysics, Academia Sinica, No. 1, Sec. 4, Roosevelt Road, Taipei 10617, Taiwan}$
$\newcommand{\TKU}{\label{TKU} Department of Physics, Tamkang University, No.151, Yingzhuan Rd., Tamsui Dist., New Taipei City 251301, Taiwan}$
$\newcommand{\PSMA}{\label{PSMA} Penn State Mont Alto, 1 Campus Drive, Mont Alto, PA  17237, USA}$
$\newcommand{\ILL}{\label{ILL} ILL}$
$\newcommand{\stromlo}{\label{stromlo} Research School of Astronomy and Astrophysics, Australian National University, Mt Stromlo Observatory, Weston Creek, ACT 2611, Australia}$
$\newcommand{\UCatolica}{\label{UCatolica} Instituto de Astronomía, Universidad Católica del Norte, Av. Angamos 0610, Antofagasta, Chile}$
$\newcommand{\UT}{\label{UT} McDonald Observatory, The University of Texas at Austin, 1 University Station, Austin, TX 78712-0259, USA}$
$\newcommand{\Vanderbilt}{\label{Vanderbilt} Department of Physics and Astronomy, Vanderbilt University, VU Station 1807, Nashville, TN 37235, USA}$
$\newcommand{\UNF}{\label{UNF} Department of Physics, University of North Florida, 1 UNF Dr. Jacksonville FL 32224}$
$\newcommand{\NAOC}{\label{NAOC} Chinese Academy of Sciences South America Center for Astronomy, National Astronomical Observatories, CAS, Beijing 100101, China}$
$\newcommand{\CASA}{\label{CASA} Center for Astrophysics and Space Astronomy, University of Colorado, 389 UCB, Boulder, CO 80309-0389, USA}$
$\newcommand{\UNAM}{\label{UNAM} Universidad Nacional Autónoma de México, Instituto de Astronomía, AP 106, Ensenada 22800, BC, México}$
$\newcommand{\UDP}{\label{UDP} Instituto de Estudios Astrofísicos, Facultad de Ingeniería y Ciencias, Universidad Diego Portales, Av. Ejército Libertador 441, Santiago, Chile}$
$\newcommand{\Steward}{\label{Steward} Steward Observatory, University of Arizona, 933 N. Cherry Ave., Tucson, AZ 85721-0065, USA}$
$\newcommand{\APO}{\label{APO} Apache Point Observatory and New Mexico State University, P.O. Box 59,$
$Sunspot, NM 88349-0059, USA}$
$\newcommand{\UNAMCU}{\label{UNAMCU} Universidad Nacional Autónoma de México, Instituto de Astronomía, AP 70-264, CDMX 04510, México}$
$\newcommand{\UWash}{\label{UWash}Department of Astronomy, University of Washington, Seattle, WA, 98195}$
$\newcommand{Ç}{\label{CC}Department of Physics, Colorado College, Colorado Springs, CO 80903}$
$\newcommand{\Utah}{\label{Utah}Department of Physics and Astronomy, University of Utah, 115 S. 1400 E., Salt Lake City, UT 84112, USA}$
$\newcommand{\UConcepcion}{\label{UConcepcion}Departamento de Astronomía, Universidad de Concepción, Casilla 160-C, Concepción, Chile}$
$\newcommand{\FCLA}{\label{FCLA}Franco-Chilean Laboratory for Astronomy, IRL 3386, CNRS and Universidad de Chile, Santiago, Chile}$
$\newcommand{\Oklahoma}{\label{Oklahoma}Homer L. Dodge Department of Physics and Astronomy, University of Oklahoma, Norman, OK 73019, USA}$
$\newcommand{\UIUC}{\label{UIUC}Department of Astronomy, University of Illinois, Urbana, IL 61801, USA}$
$\newcommand{\Harvard}{\label{Harvard}Harvard-Smithsonian Center for Astrophysics, Cambridge, MA 02138, USA}$
$\newcommand{\caltech}{\label{caltech}Department of Astronomy, California Institute of Technology, Pasadena, CA 91125, USA}$
$\newcommand{\pyf}{\texttt{pyFIT3D}\xspace}$
$\newcommand{\pyp}{\texttt{pyPipe3D}\xspace}$</div>



<div id="title">

# The SDSS-V Local Volume Mapper (LVM): Data Analysis Pipeline

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.09729-b31b1b.svg)](https://arxiv.org/abs/2411.09729)<mark>Appeared on: 2024-11-18</mark> -  _35 pages, 17 figures, 8 tables, accepted for publication in AJ_

</div>
<div id="authors">

S. F. Sánchez, et al. -- incl., <mark>K. Kreckel</mark>

</div>
<div id="abstract">

**Abstract:** We introduce the Data Analysis Pipeline (DAP) for the Sloan Digital Sky Survey V (SDSS-V) Local Volume Mapper (LVM) project, referred to as the LVM-DAP. ${ We outline our methods for recovering both stellar and emission line components from the optical integral field spectroscopy, highlighting the developments and changes implemented to address specific challenges of the data set.}$ The observations from the LVM project are unique because they cover a wide range of physical resolutions, from approximately 0.05 pc to 100 pc, depending on the distance to the targets. This, along with the varying number of stars sampled in each aperture (ranging from ${ zero, just one of a few, }$ to thousands), presents challenges in using previous spectral synthesis methods and interpreting the spectral fits. We provide a detailed explanation of how we model the stellar content and separate it from the ionized gas emission lines.  To assess the accuracy of our results, we compare them with both idealized and more realistic simulations, highlighting the limitations of our methods. We find that ${ the DAP}$ robustly correct for stellar continuum features and recover emission line parameters (e.g. flux, equivalent width, systemtic velocity and velocity dispersion) ${ with a precision and accuracy that fulfill the requirements of the primary goal of the analysis. In addition,}$ the recovered stellar parameters are reliable for single stars, the recovery of integrated populations is less precise. We conclude with a description of the data products we provide, instructions for downloading and using our software, and a showcase illustrating the quality of the data and the analysis on a deep exposure taken on the Huygens region at the center of the Orion Nebula.

</div>

<div id="div_fig1">

<img src="tmp_2411.09729/./figures/m_sel-ORION.png" alt="Fig15" width="100%"/>

**Figure 15. -** Integrated spectrum of the central 5'$\times$6' region of the Orion Nebula (black solid line), together with the best-fitted stellar model (red solid line), the model of the strongest emission lines fitted with Gaussian functions (blue solid line), and the residual after subtraction of the stellar model (cyan solid line), and the combination of both models, including a final correction to the shape of the residual (yellow solid line). The first three panels from top to bottom present the wavelength ranges covered by the blue (1st row), green (2nd row), and infrared (3rd row) arms that comprise the LVM spectrograph. Shaded regions correspond to the overlapping regimes between them. Like in the case of figures \ref{fig:lvmsim} and \ref{fig:dapsim}, the insets in the two bottom rows show a set of zoom-ins at certain particular wavelength ranges. The panels in the 4th row illustrate the quality of the modeling of the strongest emission lines, including the same lines already shown in Fig. \ref{fig:lvmsim} and \ref{fig:dapsim}. Finally, the last row of panels includes three wavelength ranges (3651-3726 Å, 4801-4999 Å, and 8151-8299 Å) that illustrate certain particularities of the spectrum discussed in the text. Three additional panels present a zoom around the three auroral lines most frequently used to determine the electron temperature in ionized nebulae: $\oi$ii 4363, $\nii$ 5755, and $\sii$i 6312. The flux ranges in these three last insets are the same to facilitate the comparison between the lines. In the particular case of this very deep spectrum, all three are clearly detected. (*fig:O_spec*)

</div>
<div id="div_fig2">

<img src="tmp_2411.09729/./figures/Scene2_sim_in.png" alt="Fig9.1" width="33%"/><img src="tmp_2411.09729/./figures/Scene2_sim_out.png" alt="Fig9.2" width="33%"/><img src="tmp_2411.09729/./figures/m_fit_Scene1.png" alt="Fig9.3" width="33%"/>

**Figure 9. -** Upper-panels: Spatial distribution of the input (left panel) and recovered (right panel) fluxes of the $\oi$ii 5007 (blue), H$\alpha$(green) and $\nii$ 6583 (red) emission lines in an arbitrary scale. Lower-panel: Example of the spectral modelling, showing the spectrum integrated across the Field-of-View (FoV) of the simulated IFS dataset (black solid line), together the best model for the stellar component (red solid line) and the set of strong ionized emission lines (blue solid line). The residuals of the subtraction of the stellar model (cyan solid line), and both models (yellow solid line) are also included. All the spectra are presented in the original flux intensities (middle panel), and normalized to one at 5000 Å (bottom panel). The orange vertical marks indicate the wavelengths at which the algorithm would attempt to extract the properties of a predefined set of emission lines using a weighted moment analysis, as described in the text. Shaded regions correspond to the overlapping regimes between the three arms of the spectrographs. The panel insets show zoom-ins in the wavelength range covered by a set of strong emission lines, including the $\oi$i doublet, H$\delta$, H$\gamma$, $\Hb$, $\oi$ii, $\Ha$, $\nii$, $\sii$, and the $\sii$i 9069, 9531 doublet.  (*fig:lvmsim*)

</div>
<div id="div_fig3">

<img src="tmp_2411.09729/./figures/Rosetta_sim_in.png" alt="Fig10.1" width="33%"/><img src="tmp_2411.09729/./figures/Rosetta_sim_out.png" alt="Fig10.2" width="33%"/><img src="tmp_2411.09729/./figures/m_fit_00006109_cl_SN1_342_5_5_20.0.png" alt="Fig10.3" width="33%"/>

**Figure 10. -** Similar figure as the one shown in Fig. \ref{fig:lvmsim}, adopting the same nomenclature, symbols and color schemes, for the realistic simulation performed using one LVM pointing on the Rosetta nebula. White hexagons inside the IFU corresponds to broken and low-transmission fibers in the input data. (*fig:dapsim*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.09729"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

247  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
